### CS4102 - Geometric Foundations of Data Analysis I
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

# Week 12: Voronoi Cells aka Catchment Areas.

* How to construct (plot) the Voronoi diagram of a given set of points in $\mathbb{R}^2$.

![voronoi](https://upload.wikimedia.org/wikipedia/commons/thumb/5/54/Euclidean_Voronoi_diagram.svg/573px-Euclidean_Voronoi_diagram.svg.png)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

###  Bisectors

* Here are some points to play with:

In [ ]:
pp = np.random.rand(4, 2)
pp

* `plot` expects $x$-coordinates, followed by $y$-coordinates.
* Thats column $0$ of `pp`, followed by column $1$ ...

In [ ]:
plt.plot(pp[:,0], pp[:,1], 'ro')

* ... or `pp` transposed and unpacked with the list unpacking operator `*`

In [ ]:
[*pp.T]

In [ ]:
plt.plot(*pp.T, 'ro')

* Let's try and construct (i.e. plot) the bisector of the first two points, $A$ and $B$.

In [ ]:
A = pp[0]
B = pp[1]
A, B

* The bisector of $A$ and $B$ is the line $\ell$ through the midpoint $M = \frac12(A + B)$ with slope $m = -1/m'$, where $m' = \frac{a_1 - b_1}{a_0 - b_0}$ is the slope of the line through $A = (a_0, a_1)$ and $B = (b_0, b_1)$.
* It follows that the equation of $\ell$ is $y = mx + c$, where $c = m_1 - m\, m_0$ if $M = (m_0, m_1)$. 

In [ ]:
M = 1/2 * (A + B)
AmB = A - B
m = - AmB[0]/AmB[1]
m

In [ ]:
c = M[1] - m * M[0]
c

* `plot` will produce a line segment if it is given two points as argument.
* Here, as $0 \leq x \leq 1$, we can plot $\ell$ as the line connecting the points $(0, y_0)$ and $(1, y_1)$ on $\ell$.
* And since the $x$-values default to $0$ and $1$ anyway, it suffices to supply $y_0 = c$ and $y_1 = m + c$.
* (That's quick and dirty rather than sustainable - but for now we don't know yet whether this needs to be sustained ...)

In [ ]:
ys = np.array([0, m]) + c
ys

In [ ]:
plt.plot(*pp[:2].T, 'ro')
plt.plot(ys)

* The line doesn't quite look perpendicular. That's because there are different scales on $x$ and $y$-axes.

In [ ]:
def bisector(A, B):
    M = 1/2 * (A + B)
    AmB = (A - B)
    m = - AmB[0]/AmB[1]
    c = M[1] - m * M[0]
    return np.array([0, m]) + c

In [ ]:
bisector(A, B)

In [ ]:
plt.xlim(0,1)
plt.ylim(0,1)
plt.plot(*np.array([A, B]).T, 'ro')
plt.plot(bisector(A, B))

* Now, let's plot all the points, and all the bisectors!  How many bisectors are there?

In [ ]:
plt.xlim(0,1)
plt.ylim(0,1)
plt.plot(*pp.T, 'ro')
for i, A in enumerate(pp):
    for j in range(i):
        B = pp[j]
        plt.plot(bisector(A, B))

* Very often, three lines seem to intersect in a single point.  That cannot be a coincidence ...

### Circumcircle

* A formula for the centre and radius of the circumcircle of (a triangle specified by) three points $A$, $B$, and $C$ can be found on [wikipedia](https://en.wikipedia.org/wiki/Circumscribed_circle#Circumcircle_equations).
* Let's look at the first three points in our list `pp`.

In [ ]:
A, B, C = pp[:3]
A, B, C

In [ ]:
plt.plot(*(pp[:3].T), 'ro')

* Shifting all points by $-A$ (so that $A$ becomes the origin $O = A - A$) simplifies the calculations

In [ ]:
B1 = B - A
C1 = C - A
B1, C1

* We need the squared lengths $b' = \|B'\|^2$ and $c' = \|C'\|^2$ of the shifted vectors

In [ ]:
b1 = B1 @ B1
c1 = C1 @ C1
b1, c1

* A rotation $R$ about $90^{\circ}$ also plays a role.

In [ ]:
rot = np.array([[0, -1], [1, 0]])
rot

In [ ]:
B1 @ rot

* Now, the circumcentre of $O$, $A'$ and $B'$ is
\\[
  U' = \frac1{d'} (b' C' - c' B').R
\\]
where $d' = 2 B'.(C'.R)^T$.
* And the radius is simply $r = \| U' \|$

In [ ]:
d1 = 2 * np.dot(B1, C1 @ rot)
d1

In [ ]:
U1 = (b1 * C1 - c1 * B1) @ rot / d1
U1

In [ ]:
r = np.sqrt(U1 @ U1)
r

* The circumcentre of $A$, $B$ and $C$ lies at $U' + A$.

In [ ]:
U1 + A

In [ ]:
circle = plt.Circle(U1 + A, r, color= 'm', fill=False, alpha=0.5)
plt.plot(*(pp[:3].T), 'ro')
plt.gca().add_patch(circle)

* Spot on!

In [ ]:
def circumcircle(A, B, C):
    B1, C1 = [B, C] - A
    b1 = B1 @ B1
    c1 = C1 @ C1
    rot = np.array([[0, -1], [1, 0]])  
    d1 = 2 * np.dot(B1, C1 @ rot)
    
    U1 = (b1 * C1 - c1 * B1) @ rot / d1
    r = np.sqrt(U1 @ U1)
    return U1 + A, r

In [ ]:
circumcircle(A, B, C)

* Let's plot all the points and all their circumcentres. How many circumcentres are there?

In [ ]:
plt.plot(*pp.T, 'ro')
circles = []
for i, A in enumerate(pp):
    for j in range(i):
        B = pp[j]
        for k in range(j):
            C = pp[k]
            centre, radius = circumcircle(A, B, C)
            circles.append({ "c" : centre, "r" : radius })
            circle = plt.Circle(centre, radius, alpha=0.2)
            plt.plot(centre[0], centre[1], 'kx')
            plt.gca().add_patch(circle)

In [ ]:
circles

* Theorem 7.4 in the book "Computational Geometry" by de Berg et al. says that a point $q$ is a vertex of the Voronoi diagram of a collection $P$ of points in $\mathbb{R}^2$ if the largest circle $C_P(q)$ with centre $q$ that does not contain any points of $P$ in its interior has at least $3$ points of $P$ on its boundary.

* Let's see, for each circle, how many of the points are inside.
* First, focusing on the first circle with centre $C$ and radius $r$, we compute, for each point $A$, the difference between the distance $d(A, C)$ and the radius $r$ (or rather their squares).

In [ ]:
c = circles[2]
for p in pp:
    d = p - c["c"]
    print(d @ d - c["r"]**2)

* Hmm.  When working with floating point numbers, 'zero' and 'being equal' are somewhat fuzzy concepts.

In [ ]:
from math import isclose
c = circles[2]
for p in pp:
    d = p - c["c"]
    print(isclose(d @ d, c["r"]**2))

In [ ]:
c = circles[1]
for p in pp:
    d = p - c["c"]
    print(not isclose(d @ d, c["r"]**2) and d @ d < c["r"]**2)

In [ ]:
plt.xlim(0,1)
plt.ylim(0,1)
plt.plot(*pp.T, 'ro')
circle = plt.Circle(c["c"], c["r"], alpha=0.2)
plt.plot(*c["c"], 'kx')
plt.gca().add_patch(circle)

* Let's wrap this up in a function

In [ ]:
def points_in_circle(points, centre, radius):
    found = []
    for p in points:
        d = p - centre
        dd = d @ d
        rr = radius**2
        if not isclose(dd, rr) and dd < rr:
            found.append(p)
    return found

In [ ]:
points_in_circle(pp, circles[0]["c"], circles[0]["r"])

In [ ]:
points_in_circle(pp, circles[1]["c"], circles[1]["r"])

* Now we can exclude those circumcircles whose interior contains points.

In [ ]:
plt.plot(*pp.T, 'ro')
for i, A in enumerate(pp):
    for j in range(i):
        B = pp[j]
        for k in range(j):
            C = pp[k]
            centre, radius = circumcircle(A, B, C)
            if len(points_in_circle(pp, centre, radius)) == 0:
                circle = plt.Circle(centre, radius, alpha=0.2)
                plt.plot(centre[0], centre[1], 'kx')
                plt.gca().add_patch(circle)

* Let's wrap this up as a function

In [ ]:
def plot_points_and_circles(points):
    plt.plot(*points.T, 'ro')
    for i, A in enumerate(points):
        for j in range(i):
            B = points[j]
            for k in range(j):
                C = points[k]
                centre, radius = circumcircle(A, B, C)
                if len(points_in_circle(points, centre, radius)) == 0:
                    circle = plt.Circle(centre, radius, alpha=0.2)
                    plt.plot(centre[0], centre[1], 'kx')
                    plt.gca().add_patch(circle)

In [ ]:
plt.xlim(1, 0)
plt.ylim(1, 0)
plot_points_and_circles(pp)

* Let's try a (slightly) bigger example

In [ ]:
ppp = np.random.rand(6, 2)
ppp

In [ ]:
plot_points_and_circles(ppp)

* Theorem 7.4 in "Computational Geometry" also says that the bisector of points $A$ and $B$ contributes an edge to the Voronoi diagram if and only if there is a point $q$ on the bisector such that the boundary of $C_P(q)$ contains $A$ and $B$ but no other point from $P$.

* It looks like we want to connect the centres of two circles if they have two of the red points in common.
* For this, we need to keep track of what points the circles belong to.
* It will be convenient to have a custom class `Circ` to support this kind of information.
* Here, we can implement the special function `__and__` in such a way that the "intersection" `C1 & C2` (python uses `&` for set intersection) computes the set of points that `C1` and `C2` have in common.

In [ ]:
class Circ:
    """a class for circles"""
    
    def __init__(self, centre, radius, poss):
        self.centre = centre
        self.radius = radius
        self.poss = poss
        
    def __repr__(self):
        return f"Circ({self.centre}, {self.radius}, {self.poss})"
        
    def __and__(self, other):
        return self.poss & other.poss

In [ ]:
c0 = Circ((0,0), 1, {1,2,3})
c1 = Circ((0,0), 1, {2,3,4})
c0 & c1

In [ ]:
len(c0 & c1)

In [ ]:
c0

* Here's a function that plots 1. all the given point, 2. all eligible circumcircles, 3. the lines connecting centers of circles intersection in two points, and 4. the centers of all eligible circles. 

In [ ]:
def plot_points_and_segments(points):
    
    # 1. points
    plt.plot(*points.T, 'ro')

    # 2. circles
    circles = []
    for i, A in enumerate(points):
        for j in range(i):
            B = points[j]
            for k in range(j):
                C = points[k]
                centre, radius = circumcircle(A, B, C)
                if len(points_in_circle(points, centre, radius)) == 0:
                    circles.append(Circ(centre, radius, {i, j, k}))
                    circle = plt.Circle(centre, radius, alpha=0.2)
                    plt.gca().add_patch(circle)

    # 3. edges
    for i, A in enumerate(circles):
        for j in range(i):
            B = circles[j]
            if len(A & B) == 2:
                plt.plot(*np.array([A.centre, B.centre]).T, 'k')

    # 4. vertices
    for A in circles:
        plt.plot(*A.centre, 'y.')

    return circles

* Test drive:

In [ ]:
plot_points_and_segments(pp)

In [ ]:
circles = plot_points_and_segments(ppp)

* And finally, a large example

In [ ]:
pppp = np.random.rand(80, 2)

In [ ]:
plt.figure(figsize=(12,12))
plt.xlim(0,1)
plt.ylim(0,1)
circles = plot_points_and_segments(pppp)

* This is certainly not the most efficient method. But it's one that I can explain in less then 1 hour.

## Exercises

1. Find (compute) the missing edges (halflines).
2. Implement the above procedure with an abstract notion of distance.
3. Compute Voronoi diagrams for the above sets of points using the Taxicab distance.